In [1]:
from datasets import load_dataset

ds = load_dataset("nyu-mll/glue", "stsb", split="test")

In [5]:
import pandas as pd

test_dataset = pd.DataFrame(ds)
test_dataset.head()

,sentence1,sentence2,label,idx
0,A girl is styling her hair.,A girl is brushing her hair.,-1.0,0
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,-1.0,1
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,-1.0,2
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,-1.0,3
4,A man is playing a harp.,A man is playing a keyboard.,-1.0,4


In [6]:
test_dataset = test_dataset.drop(columns=["label", "idx"])
test_dataset = test_dataset.rename(columns={"sentence1" : "input",
                                            "sentence2" : "reference_sentence"})

test_dataset.head()

,input,reference_sentence
0,A girl is styling her hair.,A girl is brushing her hair.
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,A man is playing a harp.,A man is playing a keyboard.


In [7]:
test_dataset.to_csv("../resource/result-base.csv", index=False)

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(len(tokenizer))

50257


In [4]:
tokenizer.add_special_tokens({"sep_token": "[SEP]"})
print(len(tokenizer))

50258


In [1]:
from tutorial.gpt2.dataset import STSBDataloader

import pytorch_lightning as pl
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
import torch
from torch import nn
import evaluate
import bert_score
import pandas as pd


class HGModel(pl.LightningModule):
    def __init__(self, tokenizer, model_name, lr: float = 1e-5):
        super().__init__()
        self.save_hyperparameters()

        self.tokenizer = tokenizer
        self.model = GPT2LMHeadModel.from_pretrained(model_name, cache_dir="/home/eahc00/.cache/huggingface/transformers")
        self.model.resize_token_embeddings(len(self.tokenizer))

        self.bleu_scorer = evaluate.load("bleu")
        self.rougeL_scorer = evaluate.load("rouge")
        self.bert_scorer = bert_score.BERTScorer(lang="en", rescale_with_baseline=False,device='cuda:1')

        self.valid_preds, self.valid_targets = [], []
        self.test_preds, self.test_targets = [], []

    def forward(self, input_ids, labels=None):
        outputs = self.model(input_ids=input_ids, labels=labels)
        return {"logits": outputs.logits, "loss": outputs.loss}

    def training_step(self, batch, batch_idx):
        inputs, labels = batch["input_ids"], batch["label_ids"]
        outputs = self(inputs, labels=labels)
        loss = outputs["loss"]
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch["input_ids"], batch["label_ids"]
        outputs = self(inputs, labels=labels)
        loss = outputs["loss"]

        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        inputs, labels = batch["input_ids"], batch["label_ids"]
        outputs = self(inputs, labels=labels)
        loss = outputs["loss"]

        # -----------------------------
        # 1) S1만 추출 (SEP 전까지)
        # -----------------------------
        sep_token_id = self.tokenizer.convert_tokens_to_ids(self.tokenizer.sep_token)

        s1_inputs = []
        for input_seq in inputs:
            # SEP 인덱스 찾기
            sep_idx = (input_seq == sep_token_id).nonzero(as_tuple=True)[0].item()
            # SEP 포함하여 S1 추출
            s1 = input_seq[: sep_idx + 1]
            s1_inputs.append(s1)

        # -----------------------------
        # 2) S1으로부터 S2 생성
        #    (배치 내 각 샘플별로 generate)
        # -----------------------------
        generated_s2 = []
        for s1 in s1_inputs:
            # shape: [1, seq_len]
            generated_seq = self.model.generate(
                s1.unsqueeze(0),  # 모델에 들어갈 수 있도록 batch 차원 추가
                do_sample=True,
                top_k=50,
                top_p=0.9,
                num_return_sequences=1,
                max_new_tokens=50,
                eos_token_id=self.tokenizer.eos_token_id,
            )
            generated_s2.append(generated_seq)

        # -----------------------------
        # 3) 예측된 S2 문장 디코딩
        #    - 각 텐서별로 decode() 적용
        # -----------------------------
        # generated_s2는 "리스트 안에 shape (1, seq_len)의 텐서들"이 들어있음
        pred_texts = []
        for seq in generated_s2:
            # seq.shape == [1, seq_len], seq[0] -> shape = [seq_len]
            text = self.tokenizer.decode(seq[0], skip_special_tokens=True)
            pred_texts.append(text)

        # -----------------------------
        # 4) 정답 S2 추출 및 디코딩
        # -----------------------------
        filtered_labels = []
        for label_seq, input_seq in zip(labels, inputs):
            sep_idx = (input_seq == sep_token_id).nonzero(as_tuple=True)[0].item()
            s2 = label_seq[sep_idx + 1 :]  # SEP 이후가 S2
            # -100 필터링 (학습 시 ignore_index)
            s2 = s2[s2 != -100]
            filtered_labels.append(s2)

        target_texts = self.tokenizer.batch_decode(filtered_labels, skip_special_tokens=True)

        # -----------------------------
        # 5) 결과 저장 (Metrics 계산용)
        # -----------------------------
        self.test_preds.extend(pred_texts)
        self.test_targets.extend(target_texts)

        return loss

    def on_test_epoch_end(self):
        logging_dict = self.compute_metrics(self.test_preds, self.test_targets, mode="test")
        self.log_dict(logging_dict, on_epoch=True, prog_bar=True)
        self.result_output(self.test_preds)

    def result_output(
        self,
        test_preds,
        read_path="/home/eahc00/tutorial/gpt2/resource/result-base.csv",
        output_path="/home/eahc00/tutorial/gpt2/resource/result.csv",
    ):
        result_df = pd.read_csv(read_path)
        result_df["predicted_label"] = test_preds
        result_df.to_csv(output_path, index=False)

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=self.hparams.lr)

    def compute_metrics(self, preds, targets, mode):
        references_bleu = [[t] for t in targets]  # BLEU가 multi-reference 구조를 필요로 함
        bleu_scores = {
            f"{mode}_bleu{n}_score": self.bleu_scorer.compute(
                predictions=preds, references=references_bleu, max_order=n
            )["bleu"]
            for n in range(1, 5)
        }
        rouge_score = self.rougeL_scorer.compute(predictions=preds, references=targets)["rougeL"]
        _, _, bert_score_result = self.bert_scorer.score(preds, targets)

        return {
            **bleu_scores,
            f"{mode}_rouge_score": rouge_score,
            f"{mode}_bert_score": bert_score_result.mean().item(),
        }

/home/eahc00/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from transformers import GPT2Tokenizer

from tutorial.gpt2.dataset import STSBDataloader

max_length = 512
learning_rate = 5e-5
max_epoch = 30
batch_size = 16
model_name = "gpt2"
seed = 42
gpu_id = [1]

pl.seed_everything(seed)

tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir="/home/eahc00/.cache/huggingface/transformers")
tokenizer.add_special_tokens(
    {
        "sep_token": "<sep>",
        "pad_token": "<pad>",
        "bos_token": "<|beginoftext|>",
        "eos_token": "<|endoftext|>",
    }
)

dataloader = STSBDataloader(tokenizer, batch_size, max_length)
dataloader.setup()

earlystopping_callback = EarlyStopping(monitor="val_loss", patience=2)

model = HGModel(tokenizer, model_name, lr=learning_rate)

trainer = pl.Trainer(
    accelerator="gpu",
    devices=gpu_id,
    max_epochs=max_epoch,
    callbacks=[earlystopping_callback],
)

trainer.fit(model=model, datamodule=dataloader)


model.eval()
trainer.test(model=model, datamodule=dataloader)


Seed set to 42
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/eahc00/venv/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 2: 100%|██████████| 360/360 [04:08<00:00,  1.45it/s, v_num=15, train_loss_step=1.660, val_loss=2.860, train_loss_epoch=1.190]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Testing DataLoader 0:  99%|█████████▉| 86/87 [02:45<00:01,  0.52it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Testing DataLoader 0: 100%|██████████| 87/87 [02:52<00:00,  0.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_bert_score      │    0.9072701930999756     │
│     test_bleu1_score      │    0.3099779188632965     │
│     test_bleu2_score      │    0.22626003623008728    │
│     test_bleu3_score      │    0.17396719753742218    │
│     test_bleu4_score      │    0.13589686155319214    │
│     test_rouge_score      │    0.3803941011428833     │
└───────────────────────────┴───────────────────────────┘

[{'test_bleu1_score': 0.3099779188632965,
  'test_bleu2_score': 0.22626003623008728,
  'test_bleu3_score': 0.17396719753742218,
  'test_bleu4_score': 0.13589686155319214,
  'test_rouge_score': 0.3803941011428833,
  'test_bert_score': 0.9072701930999756}]